In [1]:
import pandas as pd
from os.path import join

In [2]:
data_path = "..\\..\\Dataset-1\\dataset.csv"#join("..", "..", "Dataset-1", "selfie_dataset.txt")
image_path = "..\\..\\Dataset-1\\dataset"#join("..", "..", "Dataset-1", "selfie_dataset.txt")#join("..", "..", "Dataset-1", "images")

In [3]:
headers = [
    "image_name", "score", "partial_faces" ,"is_female" ,"baby" ,"child" ,"teenager" ,"youth" ,"middle_age" ,"senior" ,"white" ,"black" ,"asian" ,"oval_face" ,"round_face" ,"heart_face" ,"smiling" ,"mouth_open" ,"frowning" ,"wearing_glasses" ,"wearing_sunglasses" ,"wearing_lipstick" ,"tongue_out" ,"duck_face" ,"black_hair" ,"blond_hair" ,"brown_hair" ,"red_hair" ,"curly_hair" ,"straight_hair" ,"braid_hair" ,"showing_cellphone" ,"using_earphone" ,"using_mirror", "braces" ,"wearing_hat" ,"harsh_lighting", "dim_lighting"
]
df_image_details = pd.read_csv(data_path, names=headers, delimiter="\t")
df_image_details.head(5)

,image_name,score,partial_faces,is_female,baby,child,teenager,youth,middle_age,senior,...,curly_hair,straight_hair,braid_hair,showing_cellphone,using_earphone,using_mirror,braces,wearing_hat,harsh_lighting,dim_lighting
0,924967_564785023620424_952393378_a,5.509,-1,1,-1,-1,-1,1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,1168781_621798377908078_912486819_a,4.523,-1,1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,10175121_1411380085792172_287260819_a,4.522,-1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,-1
3,926438_281530385340209_706426712_a,5.170,-1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,-1
4,925447_666402733439620_2062222874_a,4.272,-1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,-1


In [4]:
df_image_details = df_image_details.sample(frac=1)

In [5]:
needed_columns = [
    'image_name',
    'is_female',
    'baby',
    'child',
    'teenager',
    'youth',
    'middle_age',
    'senior'
]

In [6]:
df_image_details = df_image_details[needed_columns]
# df_image_details = df_image_details[df_image_details.is_female != 0]
df_image_details.replace(to_replace=-1, value=0, inplace=True)

print(len(df_image_details))

7187


In [7]:
df_image_details_female = df_image_details[df_image_details.is_female == 1]
df_image_details_female = df_image_details_female.iloc[0:3000]
df_image_details.head(5)
df_image_details = df_image_details[df_image_details.is_female != 1]
print(len(df_image_details))
df_image_details = df_image_details.append(df_image_details_female)
print(len(df_image_details))

2667
5667


In [8]:
image_names = df_image_details.image_name.values.copy()
image_attrs = df_image_details[needed_columns[1:]].values.copy()

In [9]:
image_paths = [join(image_path, iname) + '.jpg' for iname in image_names]

In [10]:
image_paths_train, image_paths_test = image_paths[:-250], image_paths[-250:]
image_attrs_train, image_attrs_test = image_attrs[:-250], image_attrs[-250:]

In [11]:
len(df_image_details[df_image_details.is_female==1])

3000

In [12]:
from keras.utils import Sequence
import numpy as np
import cv2

C:\Users\joavi\Anaconda3\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


path :  C:\Users\joavi\Anaconda3\envs\py36\lib\site-packages\tensorflow\python\_pywrap_tensorflow_internal.pyd 
name :  _pywrap_tensorflow_internal


In [13]:
cv2.__version__

'3.3.1'

In [14]:
class ImageGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        # read your data here using the batch lists, batch_x and batch_y
        x = [self.read_image(filename) for filename in batch_x] 
        y = [atrributes for atrributes in batch_y]
        return np.array(x), np.array(y)
    
    def read_image(self, fname):
        im = cv2.imread(fname)
        im = cv2.resize(im, (224, 224))
        return im/255.

# Training Model

In [15]:
from keras.applications import resnet50
from keras.layers import Dense, Dropout, BatchNormalization, Activation, ReLU
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [22]:
pretrained = False
if not pretrained:
        model_rnet = load_model("resnet50.hdf5")
        for layer in model_rnet.layers:
            layer.trainable = False
        model_output = Dense(256)(model_rnet.get_layer('avg_pool').output)
        model_output = BatchNormalization()(model_output)
        model_output = ReLU()(model_output)
        model_output = Dropout(0.5)(model_output)
        
#         model_output = Dense(256)(model_output)
#         model_output = BatchNormalization()(model_output)
#         model_output = ReLU()(model_output)
#         model_output = Dropout(0.5)
        
        # model_output = Dense(512, activation='relu')(model_output)
        # model_output = Dropout(0.33)(model_output)
        # model_output = Dense(128, activation='relu')(model_output)
        # model_output = Dropout(0.2)(model_output)
        model_output = Dense(7, activation='sigmoid')(model_output)
        model = Model(inputs=[model_rnet.input], outputs=[model_output])
else:
    model = load_model("chpt-2-new-d.hdf5")

In [23]:
model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
train_gen = ImageGenerator(image_paths_train, image_attrs_train, batch_size=32)
test_gen = ImageGenerator(image_paths_test, image_attrs_test, batch_size=32)

In [25]:
train_len = len(image_paths_train)
test_len = len(image_paths_test)
train_len, test_len

(5417, 250)

In [26]:
tot = 4520 + 1025 + 796 + 1474 + 2380 + 755 + 677
class_weights = {
    0: 4520/tot , 1: 1025/tot , 2: 796/tot , 3: 1474/tot , 4: 2380/tot , 5: 755/tot , 6: 677/tot
}

# class_weights = [i/tot for i in class_weights]

In [27]:
model.fit_generator(train_gen, validation_data=test_gen, epochs=200, 
                    steps_per_epoch=train_len // 32,
                   validation_steps=10, use_multiprocessing=False, 
                   callbacks=[
                       ReduceLROnPlateau(patience=2, verbose=1),
                       ModelCheckpoint('chpt-2-new-d.hdf5', verbose=1, save_best_only=True)
                   ])

Epoch 1/200
169/169 [==============================] - 76s 452ms/step - loss: 0.4514 - acc: 0.7838 - val_loss: 0.6078 - val_acc: 0.8635

Epoch 00001: val_loss improved from inf to 0.60779, saving model to chpt-2-new-d.hdf5
Epoch 2/200
169/169 [==============================] - 74s 435ms/step - loss: 0.3394 - acc: 0.8348 - val_loss: 0.6709 - val_acc: 0.8644

Epoch 00002: val_loss did not improve from 0.60779
Epoch 3/200
 13/169 [=>............................] - ETA: 1:04 - loss: 0.3072 - acc: 0.8541

KeyboardInterrupt: 

In [40]:
n_columns = [
    'is_female',
    'baby',
    'child',
    'teenager',
    'youth',
    'middle_age',
    'senior'
]

try:
    cap = cv2.VideoCapture(0)
    while True:
        ret, im = cap.read()
        imr = cv2.resize(im, (224, 224))
        predictions = model.predict(np.expand_dims(imr, axis=0))[0]
        classes = [n_columns[i] for i, prob in enumerate(predictions) if prob > 0.6]
        if 'is_female' not in classes:
            classes.append('male')
        else:
            classes.remove('is_female')
            classes.append('female')
        text = ",".join(classes)
        im = cv2.putText(im, text, (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 0), lineType=cv2.LINE_AA) 
        cv2.imshow("Camera", im)
        k = cv2.waitKey(1)
        if k == ord('q'):
            raise KeyboardInterrupt
except KeyboardInterrupt:
    cv2.destroyAllWindows()
    cap.release()